In [8]:
import requests
import random
from dotenv import load_dotenv
import os
import time
import json


In [2]:
load_dotenv(dotenv_path="api_keys.env")
api_key = os.getenv("API_KEY_NewYorkTimes")
url = f"https://api.nytimes.com/svc/archive/v1/2020/10.json?api-key={api_key}"

response = requests.get(url)
data = response.json()
num_articles = data['response']['meta']['hits']

In [3]:
data['response']['docs'][1].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [27]:
'''Keys we need
=> abstract: longer headline 
=> source: publisher name
=> web_url: where to find the article
=> keywords: translates into topics and areas
=> pub date: for date of publication
=> Document type: used for pre_filtering by if media is an article, video, etc
=> news desk: Op ed,book review ===> needed to filter\
=> byline: gives us author'''
i = 2040

print(data['response']['docs'][i]['abstract'])
print(data['response']['docs'][i]['keywords'])
print(data['response']['docs'][i]['news_desk'])
print(data['response']['docs'][i]['document_type'])
print(data['response']['docs'][i]['byline'])

Ioannis Lagos was a leading member of the extreme-right and now-defunct Golden Dawn, which rose to political prominence in 2012 at the peak of the Greece’s financial crisis.
[{'name': 'subject', 'value': 'Neo Nazi Groups', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Golden Dawn (Greece)', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'European Parliament', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Greece', 'rank': 4, 'major': 'N'}]
Foreign
article
{'original': 'By Niki Kitsantonis', 'person': [{'firstname': 'Niki', 'middlename': None, 'lastname': 'Kitsantonis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}


In [5]:

outlets = {}
for i in range(num_articles):
    s = data['response']['docs'][i]['news_desk']
    if s in outlets:
        outlets[s] += 1
    else:
        outlets[s] = 1 

print(outlets)

{'OpEd': 297, 'Sports': 261, 'Politics': 617, 'Business': 367, 'The Learning Network': 8, 'Games': 40, 'Summary': 49, 'Corrections': 29, 'NYTNow': 114, 'Styles': 120, 'Learning': 105, 'Culture': 303, 'Science': 137, 'Foreign': 457, 'Magazine': 54, 'Real Estate': 38, 'BookReview': 131, 'Climate': 40, 'Neediest': 5, '': 157, 'Metro': 139, 'Upshot': 26, 'Arts&Leisure': 43, 'Metropolitan': 27, 'Weekend': 104, 'National': 161, 'Well': 61, 'Dining': 89, 'Washington': 257, 'Podcasts': 34, 'TStyle': 25, 'RealEstate': 69, 'Parenting': 28, 'Obits': 116, 'NewsDesk': 23, 'Letters': 58, 'Express': 130, 'Gender': 11, 'Opinion': 24, 'Society': 44, 'New York': 9, 'Briefing': 9, 'Travel': 22, 'U.S.': 182, 'SundayBusiness': 35, 'Editorial': 37, 'Books': 23, 'Test': 1, 'Health': 4, 'AtHome': 32, 'Smarter Living': 12, 'Business Day': 1, 'SpecialSections': 34, 'Universal': 1, 'The Upshot': 7, 'Food': 3, 'Technology': 7, 'Burst': 2, 'Multimedia/Photos': 2, 'Arts': 5, 'Investigative': 7, 'Video': 2, 'Admin':

In [30]:
# Define the years and months to scrape
years_to_scrape = [2020, 2021, 2022]
months_to_scrape = list(range(1, 13))
# years_to_scrape = [2021]
# months_to_scrape = [5]

# Storage for collected articles
articles = []
delay = 30 # in seconds
for year in years_to_scrape:
    for month in months_to_scrape:
        time.sleep(delay)
        url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"
        
        try:
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()
            
            data = response.json()
            docs = data.get('response', {}).get('docs', [])
            
            for doc in docs:
                # Filter by document type
                if doc.get('document_type') != 'article' or doc.get('news_desk') == "Corrections":
                    continue
                
                # Extract the required fields
                article = {
                    "abstract": doc.get("abstract"),
                    "source": doc.get("source"),
                    "web_url": doc.get("web_url"),
                    "keywords": [kw.get("value") for kw in doc.get("keywords", [])],
                    "pub_date": doc.get("pub_date"),
                    "document_type": doc.get("document_type"),
                    "news_desk": doc.get("news_desk"),
                    "authors": [" ".join([by.get("firstname", ""), by.get("lastname", "")]) for by in doc.get("byline", {}).get("person", {})]
                }
                
                # Append to the list of articles
                articles.append(article)
        
        except requests.exceptions.RequestException as e:
            print(f"An error occurred for {year}-{month}: {e}")

# Save the data to a file
with open("nytimes_articles.json", "w") as f:
    json.dump(articles, f, indent=2)

print(f"Scraping complete. {len(articles)} articles saved.")

            
            

An error occurred for 2020-9: 429 Client Error: Too Many Requests for url: https://api.nytimes.com/svc/archive/v1/2020/9.json?api-key=P5T5Og2xMQan89LizfjxslF6t2oGJYSL
An error occurred for 2020-10: 429 Client Error: Too Many Requests for url: https://api.nytimes.com/svc/archive/v1/2020/10.json?api-key=P5T5Og2xMQan89LizfjxslF6t2oGJYSL


KeyboardInterrupt: 